# XGBoost Ranker: Generate Kaggle Submission

In [1]:
import pandas as pd
import xgboost as xgb


## Step 1: Load your test set

In [3]:
# Ensure this file matches the feature set used for training (same columns, no labels)
test_imputed = pd.read_csv("/Users/nikak/Desktop/vubaby/DMT/data_DMT/test_imputed_CSV.csv")
feats = pd.read_csv("/Users/nikak/Desktop/vubaby/DMT/data_DMT/nika_test_feats_with_cf.csv")

drop_cols = ['click_bool', 'booking_bool', 'date_time', 'position', 'gross_bookings_usd']

test_feats_merged = test_imputed.merge(feats, on=['srch_id', 'prop_id', 'srch_destination_id'], how='left') 
test_features = test_feats_merged.drop(columns=drop_cols + ['srch_id', 'prop_id'], errors='ignore')
print("Test features loaded:", test_features.shape)


Test features loaded: (4959183, 69)


In [2]:
train_imputed = pd.read_csv("/Users/nikak/Desktop/vubaby/DMT/data_DMT/train_imputed_CSV.csv")
train_feats = pd.read_csv("/Users/nikak/Desktop/vubaby/DMT/data_DMT/nika_train_feats_with_cf.csv")

drop_cols = ['click_bool', 'booking_bool', 'date_time', 'position', 'gross_bookings_usd']
train_feats_merged = train_imputed.merge(train_feats, on=['srch_id', 'prop_id', 'srch_destination_id'], how='left')
train_features = train_feats_merged.drop(columns=drop_cols + ['srch_id', 'prop_id'], errors='ignore')
print("Train features loaded:", train_features.shape)




Train features loaded: (4958347, 69)


In [4]:
train_cols = set(train_features.columns)
test_cols = set(test_features.columns)

print("\nIn test but NOT in train:", test_cols - train_cols)
print("In train but NOT in test:", train_cols - test_cols)



In test but NOT in train: set()
In train but NOT in test: set()


## Step 2: Load the trained XGBoost model

In [5]:
model = xgb.Booster()
model.load_model("xgboost_model_final.json")
print("Model loaded successfully.")


Model loaded successfully.


## Step 3: Predict relevance scores for each hotel

In [6]:
dtest = xgb.DMatrix(test_features)
test_features['pred_score'] = model.predict(dtest)

print("Predictions completed.")


ValueError: feature_names mismatch: ['srch_id', 'site_id', 'visitor_location_country_id', 'visitor_hist_starrating', 'visitor_hist_adr_usd', 'prop_country_id', 'prop_id', 'prop_starrating', 'prop_review_score', 'prop_brand_bool', 'prop_location_score1', 'prop_location_score2', 'prop_log_historical_price', 'price_usd', 'promotion_flag', 'srch_destination_id', 'srch_length_of_stay', 'srch_booking_window', 'srch_adults_count', 'srch_children_count', 'srch_room_count', 'srch_saturday_night_bool', 'srch_query_affinity_score', 'orig_destination_distance', 'random_bool', 'comp1_rate', 'comp1_inv', 'comp1_rate_percent_diff', 'comp2_rate', 'comp2_inv', 'comp2_rate_percent_diff', 'comp3_rate', 'comp3_inv', 'comp3_rate_percent_diff', 'comp4_rate', 'comp4_inv', 'comp4_rate_percent_diff', 'comp5_rate', 'comp5_inv', 'comp5_rate_percent_diff', 'comp6_rate', 'comp6_inv', 'comp6_rate_percent_diff', 'comp7_rate', 'comp7_inv', 'comp7_rate_percent_diff', 'comp8_rate', 'comp8_inv', 'comp8_rate_percent_diff

## Step 4: Sort predictions and generate Kaggle submission

In [ ]:
submission = test[['srch_id', 'prop_id', 'pred_score']]
submission_sorted = submission.sort_values(by=['srch_id', 'pred_score'], ascending=[True, False])
submission_final = submission_sorted[['srch_id', 'prop_id']]

submission_final.to_csv("assignment_2/results/xgboost_submission.csv", index=False)
print("✅ Submission file saved to 'assignment_2/results/xgboost_submission.csv'")
